<H1 align="center">GeoViews: From exploratory analysis<br> to custom GIS dashboards in a few lines of Python code</H1>
<br>
<H2 align="center">FOSS4G Boston 2017</H2>
<H3 align="center">
Philipp Rudiger
</H3>
<H3 align="center">
Continuum Analytics
</H3>

<center>
<img src="./combined.png" width='550px'></img>
</center>

<img src="./anaconda_logo.png" style='align: left' width='200px'></img>

**Let's say you want to:** 

* Make it easy to explore some dataset interactively and then add custom widgets and deploy as an app.

**But then you have to:**
* Spend days of effort to get something working in a notebook
* Build an opaque mishmash of domain-specific, widget, and plotting code
* Start over from scratch whenever you need to:
    - Deploy in a standalone server
    - Visualize different aspects of your data
    - Scale up to larger (>100K) datasets

# SciPy Ecosystem

<figure>
<center>
<img src="./scipy_ecosystem.png" width='60%'>
</center>
<br>
<center><h4>Image by Jake VanderPlas</h4></center>
</figure>

# Bokeh Ecosystem

* Bokeh interactive plotting in the browser
* Supports Jupyter notebook for interactive plotting
* Includes bokeh server to deploy as standalone apps
* Scales to large datasets thanks to datashader
* Official high-level interface now based on HoloViews

<center>
<img src="./ds_hv_bokeh.png" width='60%'></img>
</center>

In [ ]:
import holoviews as hv
import geoviews as gv
import param, paramnb, parambokeh
import pandas as pd
import dask.dataframe as dd
import cartopy.crs as ccrs
import datashader as ds
import xarray as xr

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY, PlotSize
from geoviews import feature as gf

from bokeh.models import WMTSTileSource
from bokeh.tile_providers import STAMEN_TONER

tiles = {'OpenMap': WMTSTileSource(url='http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'),
         'ESRI': WMTSTileSource(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{X}/{Y}.jpg'),
         'Wikipedia': WMTSTileSource(url='https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png'),
         'Stamen Toner': STAMEN_TONER}

hv.extension('bokeh', 'matplotlib')

%opts Points [toolbar='above' xaxis=None yaxis=None]

# HoloViews/GeoViews

* HoloViews:
    * Declarative objects for instantly visualizable data
    * Supports different plotting extension including matplotlib and bokeh
    * Allows wrapping many datatypes including, pandas, xarray (NetCDF), shapefiles, geopandas etc.


* GeoViews:
    * GIS extension for HoloViews based on Cartopy for geographic projections
    * Wide range of declarative primitives
    * Support for:
        * Tile sources
        * Geographic features
        * Projections
        * Points
        * Shapefiles
        * Raster data

In [ ]:
%%output backend='matplotlib' dpi=120
%%opts Feature [projection=ccrs.Robinson()]
gf.coastline + gf.ocean + gf.ocean*gf.land*gf.coastline

In [ ]:
earthquakes = pd.read_csv('./data/earthquakes.csv', parse_dates=['Date'])
earthquake_ds = gv.Dataset(earthquakes, kdims=['Date', 'longitude', 'latitude']).sort()
earthquakes.tail()

In [ ]:
%%opts Points [width=600 height=530 tools=['hover'] color_index='mag' size_index='mag' colorbar=True size_fn=np.exp] (line_color='black')
eq_points = earthquake_ds.to(gv.Points, groupby='Date', dynamic=True).redim.range(mag=(0, 7))
gv.WMTS(tiles['Wikipedia']) * eq_points

In [ ]:
%%output backend='matplotlib' size=300
%%opts Points [xaxis=None yaxis=None tools=['hover'] color_index='mag' size_index='mag' colorbar=True size_fn=np.exp projection=ccrs.Robinson()] (edgecolor='black')
eq_points * gf.coastline * gf.borders

## XArray: Loading and plotting NetCDF data

In [ ]:
%%opts Image [width=600 height=400 xaxis=None yaxis=None] (cmap='viridis') Feature (line_color='white' line_width=2)
ds = xr.open_dataset('/Users/philippjfr/Downloads/rhum.2003.nc')
hum_ds = gv.Dataset(ds, label='Relative Humidity')
hum_img = hum_ds.to(gv.Image, ['lon', 'lat'], dynamic=True).redim.range(rhum=(0, 100))
hum_img * gf.borders * gf.coastline

In [ ]:
%%output backend='matplotlib' size=300
%%opts Image [xaxis=None yaxis=None] (cmap='viridis') Feature (edgecolor='white' linewidth=2)
hum_img * gf.borders * gf.coastline

# Dealing with large data: Datashader

* Data larger than ~100k points cannot easily be rendered in the browser
* Aggregating data into fixed size image allows us to explore huge datasets
* Thanks to Numba and Dask we can scale to billion datapoints on a laptop and many more on a cluster

<br>
<br>
<center>
<img src="./pipeline.png" width='80%'></img>
</center>

# A data science workflow

* Load a dataset to explore
* Interactive exploration in a notebook
* Customization and iterative improvement of a plot
* Distill the important visualization and add widgets for easy control
* Deploy as a standalone app/dashboard to deliver insight

## Step 1: Get some data

* Here we'll use a subset of the often-studied NYC Taxi dataset
* About 12 million points of GPS locations from taxis
* Stored in the efficient Parquet format for easy access
* Loaded into a Dask dataframe for multi-core<br>(and if needed, out of core or distributed) computation

In [ ]:
%time df = dd.read_parquet('./data/nyc_taxi.parq/').persist()
print(len(df))
df.head(2)

## Step 2: Prototype a plot in a notebook

* A text-based representation isn't very useful for big datasets like this, so we need to build a plot
* But we don't want to start a software project, so we use HoloViews:
    - Simple, declarative way to annotate your data for visualization
    - Large library of Elements with associated visual representation
    - Elements combine (lay out or overlay) easily
* And we'll want live interactivity, so we'll use a Bokeh plotting extension
* But our data is much too big for Bokeh directly, so we'll use Datashader to rasterize it first

In [ ]:
points = gv.Points(df, kdims=['pickup_x', 'pickup_y'], vdims=['passenger_count'], crs=ccrs.GOOGLE_MERCATOR)
options = dict(width=800,height=475,xaxis=None,yaxis=None,bgcolor='black')
taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=cm['fire']).opts(plot=options)
taxi_trips

In [ ]:
taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap=cm['fire']).opts(plot=options)
gv.WMTS(tiles['Wikipedia']) * taxi_trips

## Step 3: Declare your Parameters

Now that we've prototyped a nice plot, we want it to be widely sharable, with controls for safe and easy exploration. 

So the next step: declare what the intended user can change, with:

  - type and range checking
  - documentation strings
  - default values
  
The Param library allows declaring Python attributes having these features<br>(and more, such as dynamic values and inheritance).

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    plot        = param.ObjectSelector(default="pickup", objects=["pickup","dropoff"])
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    passengers  = param.Range(default=(0, 10), bounds=(0, 10), doc="""
        Filter for taxi trips by number of passengers""")

In [ ]:
paramnb.Widgets(NYCTaxiExplorer)

In [ ]:
#parambokeh.Widgets(NYCTaxiExplorer)

## Step 4: Link your Parameters to your data

Because the Parameters defined earlier are *about* a plot, it makes sense to combine the parameter and plotting declarations into a single object:

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    alpha       = param.Magnitude(default=0.75, doc="Alpha value for the map opacity")
    colormap    = param.ObjectSelector(default=cm["fire"], objects=cm.values())
    plot        = param.ObjectSelector(default="pickup",   objects=["pickup","dropoff"])
    passengers  = param.Range(default=(0, 10), bounds=(0, 10))

    def make_view(self, x_range=None, y_range=None, **kwargs):
        map_tiles = gv.WMTS(tiles['Wikipedia']).opts(style=dict(alpha=self.alpha), plot=options) 

        points = hv.Points(df, kdims=[self.plot+'_x', self.plot+'_y'], vdims=['passenger_count'])
        selected = points.select(passenger_count=self.passengers)
        taxi_trips = datashade(selected, x_sampling=1, y_sampling=1, cmap=self.colormap,
                               dynamic=False, x_range=x_range, y_range=y_range,
                               width=800, height=475)
        return map_tiles * taxi_trips

In [ ]:
explorer = NYCTaxiExplorer()
paramnb.Widgets(explorer, callback=explorer.event)
hv.DynamicMap(explorer.make_view, streams=[explorer, RangeXY()])

## Step 5: Deploy your dashboard

If you want to share your work with people who don't use Python, you'll now want to run a server with this same code.

* If you used **ParamBokeh**, deploy with **Bokeh Server**:
    - Write the above code to a file ``osm_parambokeh.py``,<br> switching to server mode when calling `Widgets`, which will return a bokeh `Document`
    - ``bokeh serve osm_parambokeh.py``

[Show app]

# Branching out

Much more ambitious apps are possible with very little additional code or effort:

* Adding additional linked or separate subplots of any type; see [holoviews.org](http://holoviews.org)
* Declaring code that runs for clicking or selecting *within* the Bokeh plot; see "streams" at [holoviews.org](http://holoviews.org)
* Using multiple sets of widgets of many different types; see [ParamNB](https://github.com/ioam/paramnb) and [ParamBokeh](https://github.com/ioam/parambokeh)
* Using datasets too big for any one machine, with [Dask.Distributed](https://distributed.readthedocs.io)

# Future work

* Jupyter Dashboards Server not currently maintained; requires older ipywidgets version
* Bokeh Server is mature and well supported, but does not currently support drag-and-drop layout like Jupyter Dashboards does
* ParamBokeh still needs some polishing and work to make it ready for widespread use; ParamNB is more mature so far
* Both ParamNB and ParamBokeh should provide more flexible widget layouts
* Let us know what you would like to see out of these tools!

Join us on our Gitter channel or file issues!

# Acknowledgements

* GeoViews development was partially funded by work with the UK MetOffice
* Thanks to Continuum Analytics for supporting FOSS
* Thanks to contributors 
* Come find us at **holoviews.org** and **geoviews.org**


<img src='./anaconda_logo.png' width=250px>